# oneM2M - Access Control

This notebook demonstrates how access control to resources can be done in oneM2M.

- Create an &lt;ACP> resource with different credentials for a new originator
- Create a second &lt;AE> resource with the new access controls policy
- Succeed to add a &lt;Container> to the second &lg;AE> resource
- Fail to update the second &lt;AE> resource

## Intitialization
The section does import necessary modules and configurations.

In [ ]:
%run init.py

## Create an &lt;ACP> Resource

Access Control Policies are used to associate access control with credentials. They define the rules to for access control to resources. Each &lt;ACP> resource has two sections:

- **pv (Privileges)** : The actual privileges defined by this policy.
- **pvs (Self-Privileges)** : This defines the privileges necessary to access and control the &lt;ACP> resource itself.

Each section has at least the following two parameters:

- **acor (accessControlOriginators)** : This list includes the Originator information. The parameter comprises a list of domain, CSE-IDs, AE-IDs, the resource-ID of a &lt;Group> resource that contains &lt;AE> or &lt;remoteCSE> as member or Role-ID.
- **acop (accessControlOperations)** : This number represents a bit-field of privileges. The following table shows the mapping:

| Value | Interpretation |
|-------|----------------|
| 1     | CREATE         |
| 2     | RETRIEVE       |
| 4     | UPDATE         |
| 8     | DELETE         |
| 16    | NOTIFY         |
| 32    | DISCOVERY      |

The following request creates a new &lt;ACP> that allows the originator *abc:xyz* only to send CREATE, RETRIEVE, NOTIFY, DELETE, and DISCOVERY requests to resources that have this &lt;ACP> resource assigned.

In [ ]:
CREATE (                                            # CREATE request
    url,
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=1'   # Content is JSON, and represents an <ACP> resource
    },
    
    # Request Body
    '''
    {
        "m2m:acp": {
            "rn":"Notebook-ACP",
            "pv": {
                "acr": [{
                    "acor": [ "abcxyz" ],
                    "acop": 59
                }]
            },
            "pvs": {
                "acr": [{
                    "acor": [ "%s" ],
                    "acop": 63
                }]
            }
        }
    }
    ''' % originator
)



## Create a second &lt;AE> Resource with the new &lt;ACP>

We now create a new &lt;AE> resource that uses the just created &lt;ACP>.

**This should succeed.**

In [ ]:
CREATE (                                           # CREATE request
    url,
    
    # Request Headers
    {
        'X-M2M-Origin' : 'C',                      # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=2'   # Content is JSON, and represents an <AE> resource
    },
    
    # Request Body
    '''
    {   
        "m2m:ae": {
            "rn": "Notebook-AE_2",
            "api": "AE",
            "acpi" : [ 
                "%s/Notebook-ACP" 
            ],
            "rr": true,
            "srv": ["3"]
        }
    }
    ''' % basename
)

## Try to Create &lt;Container> under the second &lt;AE> Resource

We will update a &lt;Container> resource under the second &lt;AE> resource with the originator of *abc:xyz*. 

**This should work** since this originator is allowed to send CREATE requests.

In [ ]:
 CREATE (                                          # CREATE request
    url + '/Notebook-AE_2',
    
    # Request Headers
    {
        'X-M2M-Origin' : "abcxyz",                 # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
    },
    
    # Request Body
    '''
    {
        "m2m:cnt": {
            "rn":"Container",
            "acpi": [
                "%s/Notebook-ACP"
            ]
        }
    }
    ''' % basename
)

## Try to Update the second &lt;AE> Resource

Now we try to update the new &lt;AE> resource (add a *lbl* attribute) with the other originator, *abc:xyz*. 

**<span style='color:red'>This should fail</span>**, since the associated &lt;ACP> doesn't allow UPDATE requests.

In [ ]:
UPDATE (                                           # UPDATE request
    url + '/Notebook-AE_2',
    
    # Request Headers
    {
        'X-M2M-Origin' : 'abcxyz',                 # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=2'   # Content is JSON, and represents an <AE> resource
    },
    
    # Request Body
    {
        "m2m:ae": {
            "lbl": [
                "test:test"
            ]
        }
    }
)